# Session 2 - Parameter Calibration

Usually, the parameters we obtain from the characterization of the cell result is simulations that are off from cycling data. Therefore, before using a model, it is good practice to carry out a "Calibration" step, where we fit a small subset of the model parameters to experimental voltage profiles of the same cell.

In this session, we will calibrate the Xu 2025 parameter set to experimental voltage curves from the same cell. 

Let's import BattMo and some other packages we will use.

In [1]:
using BattMo, GLMakie, CSV, DataFrames, Jutul

### Load the experimental data

We will calibrate our model in two steps: 
1. We will adjust the stoichiometric coefficients and maximum concentrations of the active materials, to fit a cell voltage curve at C/2.
2. We will adjust the reaction rate constants and diffusion coefficients in the active materials, to fit a cell voltage curve at 2C.

We first load the datasets.

In [26]:
expdata_05C = CSV.read("Xu2015_data/Xu_2015_voltageCurve_05C.csv", DataFrame)
expdata_1C = CSV.read("Xu2015_data/Xu_2015_voltageCurve_1C.csv", DataFrame)
expdata_2C = CSV.read("Xu2015_data/Xu_2015_voltageCurve_2C.csv", DataFrame)

Row,t,V
,Float64,Float64
1,0.0,3.20968
2,110.725,3.15128
3,246.598,3.10993
4,394.822,3.10201
5,543.047,3.10273
6,691.272,3.10037
7,839.497,3.07889
8,963.018,3.05584
9,1086.54,3.00935


### Run a simulation of the original parameters
Now we run a baseline simulation using the parameters obtained only from characterization of the cell. We load the parameter set, ensure an appropiate lower voltage limit and DRate, and run the simulation as we saw in previous tutorials.

In [3]:
cell_parameters_original = load_cell_parameters(; from_default_set = "Xu2015")
cycling_protocol = load_cycling_protocol(; from_default_set = "CCDischarge")

cycling_protocol["LowerVoltageLimit"] = 2.25
model_setup = LithiumIonBattery()

cycling_protocol["DRate"] = 0.5
sim_original = Simulation(model_setup, cell_parameters_original, cycling_protocol)
output_original = solve(sim_original);


✔️ Validation of ModelSettings passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 2 hours, 12 minutes as 163 report steps


Progress   1%|█                                          |  ETA: 0:20:31

Progress  85%|█████████████████████████████████████      |  ETA: 0:00:03

Progress 100%|███████████████████████████████████████████| Time: 0:00:16


╭────────────────┬───────────┬───────────────┬──────────╮
│ Iteration type │  Avg/step │  Avg/ministep │    Total │
│                │ 138 steps │ 138 ministeps │ (wasted) │
├────────────────┼───────────┼───────────────┼──────────┤
│ Newton         │   2.18841 │       2.18841 │  302 (0) │
│ Linearization  │   3.18841 │       3.18841 │  440 (0) │
│ Linear solver  │   2.18841 │       2.18841 │  302 (0) │
│ Precond apply  │       0.0 │           0.0 │    0 (0) │
╰────────────────┴───────────┴───────────────┴──────────╯
╭───────────────┬─────────┬────────────┬─────────╮
│ Timing type   │    Each │   Relative │   Total │
│               │      ms │ Percentage │       s │
├───────────────┼─────────┼────────────┼─────────┤
│ Properties    │  0.4048 │     0.86 % │  0.1223 │
│ Equations     │ 10.0119 │    31.14 % │  4.4052 │
│ Assembly      │  4.5821 │    14.25 % │  2.0161 │
│ Linear solve  │  0.6644 │     1.42 % │  0.2007 │
│ Linear setup  │  0.0000 │     0.00 % │  0.0000 │
│ Precond apply │  

Once the simulation completes, we can inspect the resutling voltage curves, and compare them with the experimental voltage curves.

In [27]:
#Simulation data
time_series = get_output_time_series(output_original)
simdata_time_original = time_series[:Time]
simdata_voltage_original = time_series[:Voltage]

#Plot
fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 0.5", xlabel = "Time / s", ylabel = "Voltage / V")
lines!(ax, simdata_time_original, simdata_voltage_original, label = "Simulation 0.5C: original parameters")
scatter!(ax, expdata_05C[:,1], expdata_05C[:,2], label = "Experimental data 0.5C", markersize = 20)
axislegend(position = :lb)
fig

We can see that the simulation with original parameters does not match well the experiment. Lets therefore fit some parameters to the experimental data.

# Low-rate Calibration
### Set up the low-rate calibration

We have developed a calibration function that takes as inputs the voltage and time arrays of the data, along with the initial simulation setup.

In [5]:
calibration_low_rate = VoltageCalibration(expdata_05C[:,1], expdata_05C[:,2], sim_original) #  VoltageCalibration(experimental_time, experimental_voltage, simulation)

VoltageCalibration([357.76627218934914, 715.9763313609469, 1074.1863905325445, 1432.396449704142, 1790.6065088757396, 2148.816568047337, 2507.0266272189347, 2877.5887573964496, 3223.44674556213, 3594.0088757396447, 3952.2189349112427, 4310.42899408284, 4668.639053254437, 5026.8491124260345, 5385.059171597633, 5743.2692307692305, 6101.479289940828, 6472.041420118343, 6817.899408284024, 7188.461538461537], [3.2943262673632967, 3.2638600156322126, 3.2518999695748874, 3.2446281622882482, 3.246486083133996, 3.245753135185418, 3.246253934281757, 3.2472569925301102, 3.2356583102522136, 3.2351808720466657, 3.2359284205519883, 3.237169467875278, 3.227800290612279, 3.2273140920726844, 3.2184384136276525, 3.217458716270091, 3.1992065602836877, 3.177878797019038, 2.8807910485472883, 2.179051790010771], Simulation(BattMo.run_battery, LithiumIonBattery("Setup object for a P2D lithium-ion model", {
    "RampUp" => "Sinusoidal"
    "Metadata" =>     {
        "Description" => "Default model settings f

This calibration object is a handy way to tailor the main settings needed to run a calibration: 
* Which model parameters are frozen
* Which model parameters are being fitted
* What are the minimum and maximum bounds of the parameters to be fitted
* The results of the calibration, i.e. the optimal parameters.

All paremters are forzen by default, so we now need to free those we are interested in, and apply some bounds to each to ensure they remain within expected ranges. Below, we free the stoichiometric coefficients and maximum concentrations.

In [6]:
free_calibration_parameter!(calibration_low_rate,
    ["NegativeElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC100"];
    lower_bound = 0.0, upper_bound = 1.0)
free_calibration_parameter!(calibration_low_rate,
    ["PositiveElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC100"];
    lower_bound = 0.0, upper_bound = 1.0)

# "StoichiometricCoefficientAtSOC0" at both electrodes
free_calibration_parameter!(calibration_low_rate,
    ["NegativeElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC0"];
    lower_bound = 0.0, upper_bound = 1.0)
free_calibration_parameter!(calibration_low_rate,
    ["PositiveElectrode","ActiveMaterial", "StoichiometricCoefficientAtSOC0"];
    lower_bound = 0.0, upper_bound = 1.0)

#  "MaximumConcentration" of both electrodes
free_calibration_parameter!(calibration_low_rate,
    ["NegativeElectrode","ActiveMaterial", "MaximumConcentration"];
    lower_bound = 10000.0, upper_bound = 1e5)
free_calibration_parameter!(calibration_low_rate,
    ["PositiveElectrode","ActiveMaterial", "MaximumConcentration"];
    lower_bound = 10000.0, upper_bound = 1e5)

VoltageCalibration([357.76627218934914, 715.9763313609469, 1074.1863905325445, 1432.396449704142, 1790.6065088757396, 2148.816568047337, 2507.0266272189347, 2877.5887573964496, 3223.44674556213, 3594.0088757396447, 3952.2189349112427, 4310.42899408284, 4668.639053254437, 5026.8491124260345, 5385.059171597633, 5743.2692307692305, 6101.479289940828, 6472.041420118343, 6817.899408284024, 7188.461538461537], [3.2943262673632967, 3.2638600156322126, 3.2518999695748874, 3.2446281622882482, 3.246486083133996, 3.245753135185418, 3.246253934281757, 3.2472569925301102, 3.2356583102522136, 3.2351808720466657, 3.2359284205519883, 3.237169467875278, 3.227800290612279, 3.2273140920726844, 3.2184384136276525, 3.217458716270091, 3.1992065602836877, 3.177878797019038, 2.8807910485472883, 2.179051790010771], Simulation(BattMo.run_battery, LithiumIonBattery("Setup object for a P2D lithium-ion model", {
    "RampUp" => "Sinusoidal"
    "Metadata" =>     {
        "Description" => "Default model settings f

We have a handy function to check parameter, values and bounds:

In [7]:
print_calibration_overview(calibration_low_rate)

NegativeElectrode: Active calibration parameters
┌──────────────────────────────────────────────────┬───────────────┬────────────────────┐
│                                             Name │ Initial value │             Bounds │
├──────────────────────────────────────────────────┼───────────────┼────────────────────┤
│              ActiveMaterial.MaximumConcentration │         31540 │ 10000.0 - 100000.0 │
│ ActiveMaterial.StoichiometricCoefficientAtSOC100 │      0.518738 │          0.0 - 1.0 │
│   ActiveMaterial.StoichiometricCoefficientAtSOC0 │         0.001 │          0.0 - 1.0 │
└──────────────────────────────────────────────────┴───────────────┴────────────────────┘
PositiveElectrode: Active calibration parameters
┌──────────────────────────────────────────────────┬───────────────┬────────────────────┐
│                                             Name │ Initial value │             Bounds │
├──────────────────────────────────────────────────┼───────────────┼────────────────────┤
│ 

### Solve the low-rate calibration

Solving the calibration problem is essentially an optimization problem. We adjust free parameters so to minimize the difference between a target (the data) and the prediction (the simulation result): is performed by solving the optimization problem. This makes use of the adjoint method implemented in Jutul.jl and the LBFGS algorithm.

For calibration, we minimize the squared difference between the predicted and observed voltage, summed over all time steps:  
                  $\sum_i (V_i - V_{exp,i})^2$  
where $V_i$ is the voltage from the model and $V_{exp,i}$ is the voltage from the experimental data at step $i$. This minimization uses in the background cool algorithms implemented in Jutul, the simulation engine of BattMo. 

In [8]:
solve(calibration_low_rate);
cell_parameters_calibrated_low_rate = calibration_low_rate.calibrated_cell_parameters;

Calibration: Starting calibration of 6 parameters.
It:   0 | val: 3.886e-02 | ls-its: NaN | pgrad: 6.480e+00
It:   1 | val: 1.829e-02 | ls-its: 1 | pgrad: 6.480e+00
It:   2 | val: 4.321e-03 | ls-its: 4 | pgrad: 1.097e+00
It:   3 | val: 4.278e-03 | ls-its: 2 | pgrad: 1.386e-01
It:   4 | val: 4.275e-03 | ls-its: 1 | pgrad: 2.249e-02
It:   5 | val: 4.273e-03 | ls-its: 1 | pgrad: 1.506e-02
It:   6 | val: 4.261e-03 | ls-its: 1 | pgrad: 1.534e-02
It:   7 | val: 4.242e-03 | ls-its: 1 | pgrad: 5.334e-02
It:   8 | val: 4.089e-03 | ls-its: 1 | pgrad: 7.754e-02
LBFGS: Line search unable to succeed in 5 iterations ...
LBFGS: Hessian not updated during iteration 9
It:   9 | val: 3.073e-03 | ls-its: 5 | pgrad: 2.324e-01
LBFGS: Line search unable to succeed in 5 iterations ...
LBFGS: Hessian not updated during iteration 10
It:  10 | val: 3.073e-03 | ls-its: 5 | pgrad: 2.898e-01
Calibration: Calibration finished in 102.050625 seconds.


We can use the same printing function to explore the results of the simulation

In [9]:
print_calibration_overview(calibration_low_rate)

NegativeElectrode: Active calibration parameters
┌──────────────────────────────────────────────────┬───────────────┬────────────────────┬─────────────────┬──────────┐
│                                             Name │ Initial value │             Bounds │ Optimized value │   Change │
├──────────────────────────────────────────────────┼───────────────┼────────────────────┼─────────────────┼──────────┤
│              ActiveMaterial.MaximumConcentration │         31540 │ 10000.0 - 100000.0 │         22039.3 │  -30.12% │
│ ActiveMaterial.StoichiometricCoefficientAtSOC100 │      0.518738 │          0.0 - 1.0 │        0.546901 │    5.43% │
│   ActiveMaterial.StoichiometricCoefficientAtSOC0 │         0.001 │          0.0 - 1.0 │       0.0263285 │ 2532.85% │
└──────────────────────────────────────────────────┴───────────────┴────────────────────┴─────────────────┴──────────┘
PositiveElectrode: Active calibration parameters
┌──────────────────────────────────────────────────┬───────────────┬─

### Compare the results of the calibration against the experimental data

We can now use the optimized parameters to run a new simulation, and compare the results to the experimental data for the 0.5C discharge curve.

In [30]:
#Setup and run simulation
sim_calibrated_low_rate = Simulation(model_setup, cell_parameters_calibrated_low_rate, cycling_protocol)
output_calibrated_low_rate = solve(sim_calibrated_low_rate);

#Get simulation data
time_series = get_output_time_series(output_calibrated_low_rate)
simdata_time_calibrated_low_rate = time_series[:Time]
simdata_voltage_calibrated_low_rate = time_series[:Voltage]

#Plot
fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 0.5")
lines!(ax, simdata_time_original, simdata_voltage_original, label = "Simulation 0.5C: Original parameters")
lines!(ax, simdata_time_calibrated_low_rate, simdata_voltage_calibrated_low_rate, label = "Simulation 0.5C: after low rate calibration")
scatter!(ax, expdata_05C[:,1], expdata_05C[:,2], label = "Experimental data 0.5C", markersize = 20)
axislegend(position = :lb)
fig

✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 2 hours, 12 minutes as 163 report steps


Progress  38%|█████████████████                          |  ETA: 0:00:00

Progress  83%|████████████████████████████████████       |  ETA: 0:00:00

Progress 100%|███████████████████████████████████████████| Time: 0:00:00


╭────────────────┬───────────┬───────────────┬──────────╮
│ Iteration type │  Avg/step │  Avg/ministep │    Total │
│                │ 145 steps │ 145 ministeps │ (wasted) │
├────────────────┼───────────┼───────────────┼──────────┤
│ Newton         │   2.13103 │       2.13103 │  309 (0) │
│ Linearization  │   3.13103 │       3.13103 │  454 (0) │
│ Linear solver  │   2.13103 │       2.13103 │  309 (0) │
│ Precond apply  │       0.0 │           0.0 │    0 (0) │
╰────────────────┴───────────┴───────────────┴──────────╯
╭───────────────┬──────────┬────────────┬──────────╮
│ Timing type   │     Each │   Relative │    Total │
│               │       μs │ Percentage │       ms │
├───────────────┼──────────┼────────────┼──────────┤
│ Properties    │ 307.1214 │    35.27 % │  94.9005 │
│ Equations     │ 126.3412 │    21.32 % │  57.3589 │
│ Assembly      │  54.6229 │     9.22 % │  24.7988 │
│ Linear solve  │ 127.9854 │    14.70 % │  39.5475 │
│ Linear setup  │   0.0000 │     0.00 % │   0.0000 │
│

# High-rate Calibration
### Set up the high-rate calibration

The second calibration is performed against the 2.0C discharge curve. In the same manner as for the first discharge curve, we set up a set of parameters to calibrate against experimental data. The parameters are:

 - The reaction rate constant of both electrodes
 - The diffusion coefficient of both electrodes

The calibration this time starts from the parameters calibrated at 0.5C, so we use the `cell_parameters_calibrated_05C` from the first `solve` to run a new simulation at 2C:

In [32]:
#Update cycling protocol to run at 2C
cycling_protocol2 = deepcopy(cycling_protocol)
cycling_protocol2["DRate"] = 2.0

#Solve simulation with parameters calibrated at 05C but running a 2C discharge protocol
sim_calibrated_low_rate = Simulation(model_setup, cell_parameters_calibrated_low_rate, cycling_protocol2)
output_calibrated_low_rate = solve(sim_calibrated_low_rate);

#Get simulation data of parameters calibrated at 0.5C but run at 2C
time_series = get_output_time_series(output_calibrated_low_rate)
simdata_time_calibrated_low_rate = time_series[:Time]
simdata_voltage_calibrated_low_rate = time_series[:Voltage]

✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 33 minutes, 0.0002274 nanoseconds as 44 report steps
╭────────────────┬──────────┬──────────────┬──────────╮
│ Iteration type │ Avg/step │ Avg/ministep │    Total │
│                │ 34 steps │ 35 ministeps │ (wasted) │
├────────────────┼──────────┼──────────────┼──────────┤
│ Newton         │  3.11765 │      3.02857 │  106 (3) │
│ Linearization  │  4.11765 │          4.0 │  140 (3) │
│ Linear solver  │  3.08824 │          3.0 │  105 (2) │
│ Precond apply  │      0.0 │          0.0 │    0 (0) │
╰────────────────┴──────────┴──────────────┴──────────╯
╭───────────────┬──────────┬────────────┬─────────╮
│ Timing type   │     Each │   Relative │   

33-element Vector{Float64}:
 3.3124875389076873
 3.2433085388303105
 3.166908993822564
 3.1665792810826034
 3.1658871124047465
 3.1646415186241166
 3.1636231950334768
 3.1624122923441766
 3.1609022845963373
 3.1592496664022915
 ⋮
 3.034320092544745
 3.0215422509350316
 3.003895437967513
 2.9673826331916144
 2.930049838351993
 2.8910893295127034
 2.8137238785442333
 2.690171961121722
 2.4710930939453934

In [12]:
# sim2_0 = Simulation(model_setup, cell_parameters, cycling_protocol2)
# output2_0 = solve(sim2_0);
# time_series_2_0 = get_output_time_series(output2_0)

# t2_0 = time_series_2_0[:Time]
# V2_0 = time_series_2_0[:Voltage]

We use the simulation at 2C ran with the parameter set calibrated at 0.5C as a starting point for our new high rate calibration task. 

This time we free the reaction rate constants and diffusion coefficients, and set some boundaries for each.

In [13]:
calibration_high_rate = VoltageCalibration(expdata_2C[:,1], expdata_2C[:,2], sim_calibrated_low_rate)

free_calibration_parameter!(calibration_high_rate,
    ["NegativeElectrode","ActiveMaterial", "ReactionRateConstant"];
    lower_bound = 1e-16, upper_bound = 1e-10)
free_calibration_parameter!(calibration_high_rate,
    ["PositiveElectrode","ActiveMaterial", "ReactionRateConstant"];
    lower_bound = 1e-16, upper_bound = 1e-10)

free_calibration_parameter!(calibration_high_rate,
    ["NegativeElectrode","ActiveMaterial", "DiffusionCoefficient"];
    lower_bound = 1e-16, upper_bound = 1e-12)
free_calibration_parameter!(calibration_high_rate,
    ["PositiveElectrode","ActiveMaterial", "DiffusionCoefficient"];
    lower_bound = 1e-16, upper_bound = 1e-12)

print_calibration_overview(calibration_high_rate)

NegativeElectrode: Active calibration parameters
┌─────────────────────────────────────┬───────────────┬───────────────────┐
│                                Name │ Initial value │            Bounds │
├─────────────────────────────────────┼───────────────┼───────────────────┤
│ ActiveMaterial.DiffusionCoefficient │       3.9e-14 │ 1.0e-16 - 1.0e-12 │
│ ActiveMaterial.ReactionRateConstant │     1.764e-11 │ 1.0e-16 - 1.0e-10 │
└─────────────────────────────────────┴───────────────┴───────────────────┘
PositiveElectrode: Active calibration parameters
┌─────────────────────────────────────┬───────────────┬───────────────────┐
│                                Name │ Initial value │            Bounds │
├─────────────────────────────────────┼───────────────┼───────────────────┤
│ ActiveMaterial.ReactionRateConstant │     3.626e-11 │ 1.0e-16 - 1.0e-10 │
│ ActiveMaterial.DiffusionCoefficient │      1.25e-15 │ 1.0e-16 - 1.0e-12 │
└─────────────────────────────────────┴───────────────┴───────────

### Solve the high-rate calibration problem

In [14]:
cell_parameters_calibrated_high_rate, = solve(calibration_high_rate);
print_calibration_overview(calibration_high_rate)

Calibration: Starting calibration of 4 parameters.
It:   0 | val: 4.731e-02 | ls-its: NaN | pgrad: 3.418e+01
It:   1 | val: 3.674e-03 | ls-its: 1 | pgrad: 3.417e+01
It:   2 | val: 2.755e-03 | ls-its: 2 | pgrad: 4.283e-01
LBFGS: Line search at max step size, Wolfe conditions not satisfied for this step
LBFGS: Hessian not updated during iteration 3
It:   3 | val: 1.468e-03 | ls-its: 1 | pgrad: 2.583e-01
It:   4 | val: 1.422e-03 | ls-its: 1 | pgrad: 3.235e-01
It:   5 | val: 1.384e-03 | ls-its: 1 | pgrad: 6.237e-02
It:   6 | val: 1.323e-03 | ls-its: 1 | pgrad: 4.813e-02
It:   7 | val: 1.317e-03 | ls-its: 1 | pgrad: 9.650e-03
It:   8 | val: 1.287e-03 | ls-its: 1 | pgrad: 9.665e-03
It:   9 | val: 1.209e-03 | ls-its: 2 | pgrad: 1.520e-02
It:  10 | val: 9.666e-04 | ls-its: 2 | pgrad: 8.773e-03
It:  11 | val: 9.134e-04 | ls-its: 2 | pgrad: 1.138e-02
It:  12 | val: 9.017e-04 | ls-its: 2 | pgrad: 1.569e-02
It:  13 | val: 8.986e-04 | ls-its: 2 | pgrad: 1.506e-02
It:  14 | val: 8.977e-04 | ls-its: 

### Compare the results of the high-rate calibration against the experimental data

We compare three simulations against the experimental data:
 1. The initial simulation with the original parameters.
 2. The simulation with the parameters calibrated against the 0.5C discharge curve.
 3. The simulation with the parameters calibrated against the 0.5C and 2.0C discharge curves.

In [ ]:
# Simulation at 2C  using original parameters
sim_original_params = Simulation(model_setup, cell_parameters_original, cycling_protocol2)
output_original_params = solve(sim_original_params, accept_invalid = false);
 
time_series = get_output_time_series(output_original_params)
simdata_time_original_params = time_series[:Time]
simdata_voltage_original_params = time_series[:Voltage]


# Simulation at 2C using calibrated parameters from 2C calibration
sim_calibrated_high_rate = Simulation(model_setup, cell_parameters_calibrated_high_rate, cycling_protocol2)
output_calibrated_high_rate = solve(sim_calibrated_high_rate, accept_invalid = false);
 
time_series = get_output_time_series(output_calibrated_high_rate)
simdata_time_calibrated_high_rate = time_series[:Time]
simdata_voltage_calibrated_high_rate = time_series[:Voltage]

# Plot 2C calibrated model vs 2C experimental data
fig = Figure()
ax = Axis(fig[1, 1], title = "CRate = 2.0")
lines!(ax, simdata_time_original_params, simdata_voltage_original_params, label = "Simulation 2C: Original parameters")
lines!(ax, simdata_time_calibrated_low_rate, simdata_voltage_calibrated_low_rate, label = "Simulation 2C: after low-rate calibration")
lines!(ax, simdata_time_calibrated_high_rate, simdata_voltage_calibrated_high_rate, label = "Simulation 2C: after high-rate calibration")
scatter!(ax, expdata_2C[:,1], expdata_2C[:,2], label = "Experimental data 2C", markersize = 20)
axislegend(position = :lb)
fig

✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 33 minutes, 0.0002274 nanoseconds as 44 report steps
╭────────────────┬──────────┬──────────────┬──────────╮
│ Iteration type │ Avg/step │ Avg/ministep │    Total │
│                │ 32 steps │ 33 ministeps │ (wasted) │
├────────────────┼──────────┼──────────────┼──────────┤
│ Newton         │   3.8125 │      3.69697 │  122 (3) │
│ Linearization  │   4.8125 │      4.66667 │  154 (3) │
│ Linear solver  │  3.78125 │      3.66667 │  121 (2) │
│ Precond apply  │      0.0 │          0.0 │    0 (0) │
╰────────────────┴──────────┴──────────────┴──────────╯
╭───────────────┬──────────┬────────────┬─────────╮
│ Timing type   │     Each │   Relative │   

# Calibrated model at all CRates

We can now compare the results of the model after both low-rate and high-rate calibration against the experimental data for the 0.5C, 1.0C, and 2.0C discharge curves. 

> **Note that we did not calibrate the model for the 1.0C discharge curve, but we still obtain a good fit!!**

In [49]:
CRates = [0.5, 1.0, 2.0]
colors = Dict(0.5 => :firebrick1, 1.0 => :teal, 2.0 => :dodgerblue4)

fig = Figure()
ax = Axis(fig[1, 1], title = "Simulations vs. Experiments: after calibration")
scatter!(ax, expdata_05C[:,1], expdata_05C[:,2], label = "Experimental data 0.5C", markersize = 15, color = colors[0.5])
scatter!(ax, expdata_1C[:,1], expdata_1C[:,2], label = "Experimental data 1C", markersize = 15, color =colors[1.0])
scatter!(ax, expdata_2C[:,1], expdata_2C[:,2], label = "Experimental data 2C", markersize = 15, color =colors[2.0])

for CRate in CRates
    #Setup and run simulation
    cycling_protocol["DRate"] = CRate
    sim = Simulation(model_setup, cell_parameters_calibrated_high_rate, cycling_protocol)
    output = solve(sim, accept_invalid = false)

    #Get time series from simulation result
    time_series = get_output_time_series(output)
    t = time_series[:Time]
    V = time_series[:Voltage]

    #Plot simulation voltage response
    lines!(ax, t, V, label = "Simulation $CRate: after high-rate calibration", color = colors[CRate])
end  

axislegend(position = :lb)
fig

✔️ Validation of CellParameters passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of CyclingProtocol passed: No issues found.
──────────────────────────────────────────────────
✔️ Validation of SimulationSettings passed: No issues found.
──────────────────────────────────────────────────
Jutul: Simulating 2 hours, 12 minutes as 163 report steps


Progress  37%|████████████████                           |  ETA: 0:00:00

Progress  74%|████████████████████████████████           |  ETA: 0:00:00

Progress 100%|███████████████████████████████████████████| Time: 0:00:00


╭────────────────┬───────────┬───────────────┬──────────╮
│ Iteration type │  Avg/step │  Avg/ministep │    Total │
│                │ 151 steps │ 151 ministeps │ (wasted) │
├────────────────┼───────────┼───────────────┼──────────┤
│ Newton         │   2.15894 │       2.15894 │  326 (0) │
│ Linearization  │   3.15894 │       3.15894 │  477 (0) │
│ Linear solver  │   2.15894 │       2.15894 │  326 (0) │
│ Precond apply  │       0.0 │           0.0 │    0 (0) │
╰────────────────┴───────────┴───────────────┴──────────╯
╭───────────────┬──────────┬────────────┬──────────╮
│ Timing type   │     Each │   Relative │    Total │
│               │       μs │ Percentage │       ms │
├───────────────┼──────────┼────────────┼──────────┤
│ Properties    │ 180.8770 │    21.76 % │  58.9659 │
│ Equations     │ 169.4111 │    29.82 % │  80.8091 │
│ Assembly      │  51.7327 │     9.11 % │  24.6765 │
│ Linear solve  │ 169.0580 │    20.34 % │  55.1129 │
│ Linear setup  │   0.0000 │     0.00 % │   0.0000 │
│

Progress  75%|█████████████████████████████████          |  ETA: 0:00:00

Progress 100%|███████████████████████████████████████████| Time: 0:00:00


╭────────────────┬──────────┬──────────────┬──────────╮
│ Iteration type │ Avg/step │ Avg/ministep │    Total │
│                │ 76 steps │ 76 ministeps │ (wasted) │
├────────────────┼──────────┼──────────────┼──────────┤
│ Newton         │  2.38158 │      2.38158 │  181 (0) │
│ Linearization  │  3.38158 │      3.38158 │  257 (0) │
│ Linear solver  │  2.38158 │      2.38158 │  181 (0) │
│ Precond apply  │      0.0 │          0.0 │    0 (0) │
╰────────────────┴──────────┴──────────────┴──────────╯
╭───────────────┬──────────┬────────────┬──────────╮
│ Timing type   │     Each │   Relative │    Total │
│               │       μs │ Percentage │       ms │
├───────────────┼──────────┼────────────┼──────────┤
│ Properties    │ 171.9293 │    21.27 % │  31.1192 │
│ Equations     │ 115.1459 │    20.23 % │  29.5925 │
│ Assembly      │  50.9202 │     8.94 % │  13.0865 │
│ Linear solve  │ 255.5359 │    31.61 % │  46.2520 │
│ Linear setup  │   0.0000 │     0.00 % │   0.0000 │
│ Precond apply │  